In [ ]:
import os
from numpy import *
import json
import pandas as pd
from random import *
import math
from datetime import *
from dateutil import parser
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Get required data
From the github repository

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/XLipcak/rev-rec.git cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 1170, done.
remote: Total 1170 (delta 0), reused 0 (delta 0), pack-reused 1170
Receiving objects: 100% (1170/1170), 45.11 MiB | 14.96 MiB/s, done.
Resolving deltas: 100% (503/503), done.
Checking out files: 100% (88/88), done.


In [ ]:
%ls

cloned-repo/  drive/  sample_data/


In [ ]:
# Save OpenStack dataset
import shutil
src = "/content/cloned-repo/rev-rec-data/openstack.json"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/openstack.json"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/openstack.json'

# Process raw data 
a JSON file

In [ ]:
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/openstack.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'subProject': 'openstack/tempest', 'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d', 'changeNumber': 6139, 'timestamp': 1333428006000, 'reviewers': [{'accountId': '97', 'email': 'daryl.walleck@rackspace.com', 'name': 'Daryl Walleck', 'avatar': None}, {'accountId': '7', 'email': 'jaypipes@gmail.com', 'name': 'Jay Pipes', 'avatar': None}], 'owner': {'accountId': '2238', 'email': 'rajalakshmi.girish@hpe.com', 'name': 'rajalakshmi-ganesan', 'avatar': None}, 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}]}
{'subProject': 'openstack/tempest', 'changeId': 'I10e7116570f922ec87e23b1f880cd4b1c08c3088', 'changeNumber': 7438, 'timestamp': 1337072165000, 'reviewers': [{'accountId': '97', 'email': 'daryl.walleck@rackspace.com', 'name': 'Daryl Walleck', 'avatar': None}, {'accountId': '7', 'email': 'jaypipes@gmail.com', 'name': 'Jay Pipes', 'avatar': None}], 'owner': {'accountId': '4120', 'email': 'rohit.karajgi@ril.com', 'name': 'Rohit Karajgi', 'avatar': None}, 'filePaths': 

## Hands on with given data

In [ ]:
type(data)

list

In [ ]:
len(data)

6545

In [ ]:
# get an element of the list of data
data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'changeNumber': 6139,
 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}],
 'owner': {'accountId': '2238',
  'avatar': None,
  'email': 'rajalakshmi.girish@hpe.com',
  'name': 'rajalakshmi-ganesan'},
 'reviewers': [{'accountId': '97',
   'avatar': None,
   'email': 'daryl.walleck@rackspace.com',
   'name': 'Daryl Walleck'},
  {'accountId': '7',
   'avatar': None,
   'email': 'jaypipes@gmail.com',
   'name': 'Jay Pipes'}],
 'subProject': 'openstack/tempest',
 'timestamp': 1333428006000}

In [ ]:
# read a single feature of an element
data[0]['reviewers']

[{'accountId': '97',
  'avatar': None,
  'email': 'daryl.walleck@rackspace.com',
  'name': 'Daryl Walleck'},
 {'accountId': '7',
  'avatar': None,
  'email': 'jaypipes@gmail.com',
  'name': 'Jay Pipes'}]

## Cleaning data

In [ ]:
# Extract and save desired features from each pull request. The new desired data is new_data
new_data = []

for i in range(0, len(data)):
  elem_dict = {"changeId":data[i]['changeId'], "reviewers":data[i]['reviewers'], "owner":data[i]['owner'], "filePaths":data[i]['filePaths'], "time":data[i]['timestamp']}
  new_data.append(elem_dict)
  # print(new_data[i])

print(len(new_data))

6545


In [ ]:
new_data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}],
 'owner': {'accountId': '2238',
  'avatar': None,
  'email': 'rajalakshmi.girish@hpe.com',
  'name': 'rajalakshmi-ganesan'},
 'reviewers': [{'accountId': '97',
   'avatar': None,
   'email': 'daryl.walleck@rackspace.com',
   'name': 'Daryl Walleck'},
  {'accountId': '7',
   'avatar': None,
   'email': 'jaypipes@gmail.com',
   'name': 'Jay Pipes'}],
 'time': 1333428006000}

In [ ]:
# Remove remaining undesired features. The final desired information is saved in final_data
final_data = []

for i in range(0, len(new_data)):
  elem_dict = {"time":new_data[i]['time'], "changeId":new_data[i]['changeId'], "ownerId": new_data[i]['owner']['accountId'], "reviewersId":[], "filePaths":[]}

  for j in range(0, len(new_data[i]['reviewers'])):
    elem_dict["reviewersId"].append(new_data[i]['reviewers'][j]['accountId'])

  for j in range(0, len(new_data[i]['filePaths'])):
    elem_dict["filePaths"].append(new_data[i]['filePaths'][j]['location'])
  
  final_data.append(elem_dict)
  # print(final_data[i])

print(len(final_data))

6545


In [ ]:
final_data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'filePaths': ['tempest/tests/test_volumes_get.py'],
 'ownerId': '2238',
 'reviewersId': ['97', '7'],
 'time': 1333428006000}

In [ ]:
# change timestamp value from int to date format
for pr in final_data:
  timestamp = int(pr['time']/1000)  # removing the 3 last zero digits 
  dt_obj = datetime.fromtimestamp(timestamp) 
  pr['time'] = dt_obj

In [ ]:
final_data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'filePaths': ['tempest/tests/test_volumes_get.py'],
 'ownerId': '2238',
 'reviewersId': ['97', '7'],
 'time': datetime.datetime(2012, 4, 3, 4, 40, 6)}

In [ ]:
# sort data by time
sorted_data = sorted(final_data, key=lambda d: d['time'])

# Get the Time of available data from selected project
print("Start date:", sorted_data[0]['time'])
print("End date:", sorted_data[-1]['time'])

Start date: 2011-07-18 15:43:34
End date: 2012-05-30 21:39:57


The openstack data that we use in here, containes about 10 months information of openstack project (2011-2012)

In [ ]:
type(sorted_data)

list

In [ ]:
# Convert list to json
# It should be a string before write it to file
# "default" is added to dumps to make anything not serializable for JSON to string format (like datatime) 
json_string = json.dumps(sorted_data, default=str) 

# Save OpenStack new dataset
file = open('openstack_final.json', 'w')
file.write(json_string)
file.close()

In [ ]:
# Move OpenStack dataset to proper location
import shutil

src = "./openstack_final.json"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json'

# Approach

Should run the code each time from this part

## Main components of the approach
Extract 3 interaction metrices

### Reviewer's **expertise** model
FR matrix
as 'cal_expertise' function

In [ ]:
# Function to extract 'Expertise' metrix
# Returns P*M matrix named FR
def cal_expertise(files, all_reviewers, train):
  FR = array([[0]*len(all_reviewers)] * len(files))

  # fill the cells of the dataframe by checking PRs one by one
  for f in files:
    for pr in train:
      if f in pr['filePaths']:
        for r in pr['reviewersId']:
          FR[files.index(f)][all_reviewers.index(r)] += 1

  return FR

### Reviewer‑developer **collaboration** model
DR matrix
as 'cal_collaboration' function

In [ ]:
# Function to extract 'Collaboration' metrix
# Returns N*M matrix named DR
def cal_collaboration(all_devs, all_reviewers, train):
  DR = array([[0]*len(all_reviewers)] * len(all_devs))

  # fill the values of dataframe by checking PRs one by one
  for pr in train:
    for r in pr['reviewersId']:
      DR[all_devs.index(pr['ownerId'])][all_reviewers.index(r)] += len(pr['filePaths'])

  return DR

### **Availability** model
as 'cal_availability' function

In [ ]:
# Function to extract 'Availability' vector
# Returns 1*M matrix named A 
def cal_availability(all_reviewers, train, cur_time):

  # seven day before the time of our test PR
  week_ago = cur_time - timedelta(days=7)
  
  # Extract a list of PRs that are between time of seven days ago up to the time of test PR
  week_pr = [pr for pr in train if (week_ago < pr['time'] < cur_time)]
  # print("Num of PRs in last 7 days:", len(week_pr))

  # Extract the availability matrix named 'A' that is 1*M
  A = array([0]*len(all_reviewers))
  for r in all_reviewers:
    for pr in week_pr:
      if r in pr['reviewersId']:
        A[all_reviewers.index(r)] += len(pr['filePaths'])
  
  return A

### **File-Developer** matrix

In [ ]:
# Function to extract File-Developer matrix
# Returns N*P matrix named FD 
def cal_fileDev(all_devs, files, train):
  FD = array([[0]*len(files)] * len(all_devs))

  # Fill the dataframe with number of times each developer worked on each file
  for pr in train:
    for f in pr['filePaths']:
      if f in files:
        FD[all_devs.index(pr['ownerId'])][files.index(f)] += 1

  return FD

## Fitness functions

In [ ]:
def fit_avail(A, S):
  avail = sum(dot(A, S.T))
  return divide(1, avail, where=(avail!=0))

In [ ]:
def fit_expert(FR, S):
  return sum(divide(FR, S, out=zeros_like(FR, dtype=float), where=(S!=0)))

################### Alternative #######################
# PR matrix that is file priority will set to all zeros (because there is no priority tag)
  # PR = len(files)*[0]
  # Expertise = 0
  # for k in range(0, len(files)):
  #   for i in range(0, len(all_reviewers)):
  #     if S[k][i] > 0:
  #       Expertise += ((FR[k][i] + PR[k]) / S[k][i])
  # return Expertise

In [ ]:
def fit_collab(P, M, N, DR, FD, S):
  return sum([DR[k][i]*FD[k][j] for k in range(N) for j in range(P) for i in range(M) if S[j][i]>0])

################### Alternative #######################
# Collaboration = 0
# for k in range(0, N):
#   for j in range(0, P):
#     for i in range(0, M):
#       if S[j][i]>0:
#         Collaboration += (DR[k][i]*FD[k][j])
# return Collaboration

## Genetic

### Chromosome defenition


In [ ]:
# Chromosome creation
# Reviewer-File matrix named S with shape of P*M 

def init_gene(P, M):
  S = array([[0]*M] * P)

  # assign random number to each cell of the dataframe
  # each "row" must have all the integer numbers from 0 to M (distinct numbers)
  for row in range(0, P):
    num_of_zeros = randint(0, M-1)   # we should choose atleast one reviewer (no row with all zeros)
    # print("number of zero ranks in row", row, "is:", num_of_zeros)
    rand_ranks = num_of_zeros * [0]
    rand_ranks.extend(sample(range(1, M-num_of_zeros+1), M-num_of_zeros))  # sampling without replacement
    # print("number of non-zero ranks in row", row, "is:", max(rand_ranks))
    shuffle(rand_ranks)   # Shuffle zeros with non-zero ranks
    S[row] = rand_ranks
  
  return S

### Offspring creation

In [ ]:
# Swap mutation function
def mutation(gene, mute_prob, modif_prob, P):
  r = random()
  if r < mute_prob:
    # mutation can accure at each row with some probability
    dict_rows = {}  # dictionary of selected rows for mutation and their probability
    for i in range(0, P): 
      rand = random()
      if rand < modif_prob:
        dict_rows[i] = rand   # this row added to selected rows for mutation process
    # print("rows number for swapping", dict_rows)

    if len(dict_rows) > 1:  # if two or more rows selected
      # find two rows with maximum probabilities (sort dictionary by values in descending order)
      sort_dict = sorted(dict_rows.items(), key=lambda x: x[1], reverse=True)
      r1, r2 = sort_dict[0][0], sort_dict[1][0]
      # print("selected rows for swapping:", r1, r2)
      # swapping these two rows
      gene[[r1, r2]] = gene[[r2, r1]]
    
  return gene

In [ ]:
# Function to repaire genes with infeasible ranks
# This function not change the gene if it is a feasible solution
# the repair mechanism just apply on the mixed row of the child
def repair_gene(gene):
  non_zero_vals = gene[(gene!=0)]   # non zero elements of gene matrix
  non_zero_vals.sort()
  new_vals = [i for i in range(1, len(non_zero_vals)+1)]   # new elements for repaired gene

  dic = dict(zip(new_vals, non_zero_vals))  # create a dictionary of new values to previous one
  key_list, val_list = list(dic.keys()), list(dic.values())

  # replace gene elements with repaired ones according to dictionary
  for i in range(0, len(gene)):
    if gene[i] != 0:
      position = val_list.index(gene[i])
      val_list.remove(gene[i])
      gene[i] = key_list[position]
      key_list.remove(key_list[position])

  # return repaired gene
  return gene

In [ ]:
# Single point crossover function
def crossover(gene1, gene2, cross_prob, mute_prob, modif_prob, P, M):
  childs = []
  r = random()  # random number between 0 and 1
  if r < cross_prob:
    # crossover point
    row, col = randint(0, P-1), randint(0, M-1)   
    # print("crossover point", row, col)

    # first child : (0, 0) to (row, col) from first parent and the rest from second parent
    child1 = gene1[0:row, :]
    child1 = append(append(append(child1, gene1[row, 0:col]), gene2[row, col:]), gene2[row+1:, :]).reshape((P, M))
    # second child : first part from parent2 and second part from parent1
    child2 = gene2[0:row, :]
    child2 = append(append(append(child2, gene2[row, 0:col]), gene1[row, col:]), gene1[row+1:, :]).reshape((P, M))
    # print("child1 before repair", child1)
    # print("child2 before repair", child2)

    # repair mechanism on mixed row of each child
    child1[row] = repair_gene(child1[row])
    child2[row] = repair_gene(child2[row])
    # print("child1 after repair", child1)
    # print("child2 after repair", child2)

    # mutation
    child1 = mutation(child1, mute_prob, modif_prob, P)
    # print("child1 after mutation", child1)
    child2 = mutation(child2, mute_prob, modif_prob, P)
    # print("child2 after mutation", child2)

    childs.append(child1)
    childs.append(child2)

  return childs     # it returns empty list when no crossover happend

In [ ]:
def create_new_pop(P0, pop_size, cross_prob, mute_prob, modif_prob, P, M):
  childs = []
  while len(childs) < pop_size:
    gene1 = P0[randint(0, pop_size-1)]
    # print("first parent", gene1)
    gene2 = P0[randint(0, pop_size-1)]
    # print("second parent", gene2)
    child1_2 = crossover(gene1, gene2, cross_prob, mute_prob, modif_prob, P, M)
    if len(child1_2)!=0:
      childs.append(child1_2[0])
      childs.append(child1_2[1])
    # print("childs up to now", childs)
    # print("-----------------next loop-------------------")
  return childs

### Non-domination Rank

In [ ]:
def dominate(a, b, avail_values, expert_values, collab_values):
  if ((avail_values[a] > avail_values[b] and expert_values[a] > expert_values[b] and collab_values[a] < collab_values[b])
      or (avail_values[a] >= avail_values[b] and expert_values[a] >  expert_values[b] and collab_values[a] <  collab_values[b]) 
      or (avail_values[a] >  avail_values[b] and expert_values[a] >= expert_values[b] and collab_values[a] <  collab_values[b])
      or (avail_values[a] >  avail_values[b] and expert_values[a] >  expert_values[b] and collab_values[a] <= collab_values[b])
      or (avail_values[a] >= avail_values[b] and expert_values[a] >= expert_values[b] and collab_values[a] <  collab_values[b])
      or (avail_values[a] >= avail_values[b] and expert_values[a] >  expert_values[b] and collab_values[a] <= collab_values[b])
      or (avail_values[a] >  avail_values[b] and expert_values[a] >= expert_values[b] and collab_values[a] <= collab_values[b])):
    return True   # a dominates b
  else:
    return False 

In [ ]:
def fast_non_dominated_sort(avail_values, expert_values, collab_values):
  pop = len(avail_values)   # population
  S = [[] for i in range(0, pop)]  # means who are you dominating
  n = [0 for i in range(0, pop)]   # means how many people dominate you
  fronts = [[]]  # Pareto-fornts

  # fill S and n lists for each gene
  for p in range(0, pop):
    for q in range(p+1, pop):
      if dominate(p, q, avail_values, expert_values, collab_values):   # p dominates q
        if q not in S[p]:
          S[p].append(q)
        n[q] += 1
      elif dominate(q, p, avail_values, expert_values, collab_values): # q dominates p
        n[p] += 1
        if p not in S[q]:
          S[q].append(p)

    if n[p] == 0:
      if p not in fronts[0]:
        fronts[0].append(p)
    # print("chromosome number", p, "S is:", S[p], "n is:", n[p])

  # calculate other front levels of the remaining population
  i = 0
  while(fronts[i] != []):
    Q = []
    for p in fronts[i]:
      for q in S[p]:
        n[q] -= 1
        if n[q] == 0:
          if q not in Q:
            Q.append(q)
    i = i+1
    fronts.append(Q)

  del fronts[len(fronts)-1]  # delete one last empty level added to front levels
  return fronts

### Crowding Distance

In [ ]:
def crowding_distance(avail_values, expert_values, collab_values, front):
  distances = [0.0] * len(avail_values)

  # crowd: [([obj_1, obj_2, ...], i_0), ([obj_1, obj_2, ...], i_1), ...]
  crowd = [([avail_values[f], expert_values[f], collab_values[f]], f) for f in front]
  # print("crowd is:",  crowd)

  for i in range(3):  # calculate for each fitness function
    crowd.sort(key=lambda element: element[0][i])
    # print("crowd after sort by", i, crowd)

    # After sorting, boundary solutions are assigned Inf 
    distances[crowd[0][1]] = float("Inf")
    distances[crowd[-1][1]] = float("inf")
    
    if crowd[-1][0][i] == crowd[0][0][i]:  # If objective values are same, skip this loop
      continue

    norm = float(crowd[-1][0][i] - crowd[0][0][i])   # normalization (max - min) as Denominator

    # calculate each individual's Crowding Distance of i th objective
    for prev, cur, next in zip(crowd[:-2], crowd[1:-1], crowd[2:]):
      distances[cur[1]] += (next[0][i] - prev[0][i]) / norm  # sum up the distance of ith individual along each of the objectives

    # print("distances:", distances)

  return [distances[i] for i in front]

In [ ]:
# Sort given front in descending order according to crowding distance values
def sort_front_by_dist(Availability_values, Expertise_values, Collaboration_values, front):
  crowding_distance_values = crowding_distance(Availability_values, Expertise_values, Collaboration_values, front)
  
  # create a dictionary of each member of F[i] and its corresponding crowding distance
  dic = dict(zip(front, crowding_distance_values))

  # sort dictionary by values in descending order
  sorted_dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)

  # keys of dictionary are the number of genes
  front = [sorted_dic[i][0] for i in range(0, len(sorted_dic))]  # sorted front by crowding distance values
  return front

### Genetic core

In [ ]:
def genetic(P, M, N, files, all_reviewers, all_devs, FR, DR, A, FD, pop_size, max_gen, cross_prob, mute_prob, gene_modif_prob):
  # Create initial population
  P0 = [init_gene(P, M) for i in range(0, pop_size)]
  # print("Initial population:", len(P0))
  # print(P0)

  t = 0   # generation number

  while t < max_gen:   # while stopping criteria not reached
  
    if t % 10 == 0:
      print("Generation number:", t)

    # Generating offspring population
    Q0 = create_new_pop(P0, pop_size, cross_prob, mute_prob, gene_modif_prob, P, M)
    # print("Offspring population:", len(Q0))

    R = array(P0 + Q0)     # final population is union of parents and childs
    # print("R:", R)
    total_pop = len(R)     # number of whole population contains parents and childs that is 2*pop_size (constant number to the end of algorithm)
    # print("total number of chromosomes:", total_pop)

    # Calculate fitness values for all genes in population
    Availability_values = [fit_avail(A, R[i]) for i in range(0, total_pop)]
    # print("Avails:", Availability_values)
    Expertise_values = [fit_expert(FR, R[i]) for i in range(0, total_pop)]
    # print("Expertise:", Expertise_values)
    Collaboration_values = [fit_collab(P, M, N, DR, FD, R[i]) for i in range(0, total_pop)]
    # print("Collaboration:", Collaboration_values)

    # Fast non-dominated sort
    F = fast_non_dominated_sort(Availability_values, Expertise_values, Collaboration_values)
    # print("Fronts:", F)

    P_new = []    # new parent population
    i = 0
    while len(P_new) + len(F[i]) <= pop_size:
      # Apply crowding distance on F[i]   !!!!!!!!
      # if (t+1 == max_gen):  # in last generation, sort fronts by crowding distance values to get sorted final solution
      #   F[i] = sort_front_by_dist(Availability_values, Expertise_values, Collaboration_values, F[i])
      P_new.extend(F[i])   # union of new parents and current front
      i += 1
      # print("new parents:", P_new)
    
    # Apply crowding distance on F[i] only. because this front should be splitted
    F[i] = sort_front_by_dist(Availability_values, Expertise_values, Collaboration_values, F[i])

    # The first pop_size-|P_new| elements of F_i are chosen
    parent_indexes = P_new + F[i][0:(pop_size - len(P_new))]
    # print("Selected parent numbers for next generation:", parent_indexes)

    # corresponding genes of gene numbers kept as parents for next generation
    P0 = [R[idx] for idx in parent_indexes]
    # print("Next solutions:", P0) 
    t += 1

  # Return solution set
  print("Final set of solutions:", P0)     # A set of solutions for problem
  return P0


## Run

### load final data

In [ ]:
# Load final data from drive
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'time': '2011-07-18 15:43:34', 'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612', 'ownerId': '1', 'reviewersId': ['2'], 'filePaths': ['modules/ssh/files/sshd_config', 'manifests/server.pp', 'modules/ssh/manifests/init.pp']}
{'time': '2011-07-18 16:56:22', 'changeId': 'Iea32ad3aedbee996db53655842061d14c9e3d876', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp', 'modules/jenkins_slave/files/known_hosts']}
{'time': '2011-07-18 17:14:05', 'changeId': 'Ib6af08a6af71a6e5853d958a12f63c381eafa039', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 14:37:53', 'changeId': 'Ide7975c522b6b31a23af67b135135c07d75d9bde', 'ownerId': '2', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/files/pubring.gpg', 'modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 15:12:47', 'changeId': 'I28119ac1735127812866a4b0519368a829e2e666', 'ownerId': '2', 'r

In [ ]:
# convert times to datetime format
for pr in data:
  pr['time'] = parser.parse(pr['time'])

data[0]

{'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612',
 'filePaths': ['modules/ssh/files/sshd_config',
  'manifests/server.pp',
  'modules/ssh/manifests/init.pp'],
 'ownerId': '1',
 'reviewersId': ['2'],
 'time': datetime.datetime(2011, 7, 18, 15, 43, 34)}

In [ ]:
# Number of files in whole data
all_files = set()   # make a "set" of all file paths
max_file = 0

for pr in data:
  num_file = 0
  for f in pr['filePaths']:
    all_files.add(f)
    num_file += 1
  if num_file > max_file:
    max_file = num_file

print("Total number of files in dataset:", len(all_files))
print("Maximum number of files in a PR:", max_file)

Total number of files in dataset: 11421
Maximum number of files in a PR: 1757


In [ ]:
# Number of reviewers in whole data
all_reviewers = set()   # make a set of all reviewers
for pr in data:
  for r in pr['reviewersId']:
    all_reviewers.add(r)

print("Total number of reviewers in dataset:", len(all_reviewers))

Total number of reviewers in dataset: 82


In [ ]:
# Number of developers in whole data
all_devs = set()   # make a "set" of all developers
for pr in data:
  all_devs.add(pr['ownerId'])

print("Total number of developers in dataset:", len(all_devs))

Total number of developers in dataset: 324


### split train/test sets

In [ ]:
# Split data for train/test sets
init_train = data[0:5900]     # 90% of data is for initial train set (5900 PR)
total_test = data[5900:6545]  # 10% of data will be tested in the whole examination (645 PR)

### functions

In [ ]:
def get_files(pr):
  # create a "set" of file paths in PR. we selected set to ensure not collecting repeated files 
  return {f for f in pr['filePaths']}

In [ ]:
def get_all_reviewers(train):
  # create a set of all reviewers
  all_reviewers = set()   
  for pr in train:
    for r in pr['reviewersId']:
      all_reviewers.add(r)
  return all_reviewers

In [ ]:
def get_all_devs(train):
  # create a "set" of all developers 
  return {pr['ownerId'] for pr in train}

### Experiment & Result

In [ ]:
# Function to write predicted top-k reviewers for each test into a JSON file
def save_predictions(top_13510, test_num):
  predictions = {}
  predictions['test_num'] = test_num
  predictions[1] = top_13510[0]
  predictions[3] = top_13510[1]
  predictions[5] = top_13510[2]
  predictions[10] = top_13510[3]
  with open('output.txt', 'a') as outfile:
    json.dump(predictions, outfile, indent=4)

In [ ]:
# Function for saving that predictions are true or flase
def save_true_pred(true_pred):
  with open('result.txt', 'a') as outfile:
    outfile.write(str(true_pred)+'\n')

In [ ]:
# Function to predict top-k reviewers for a test PR
# Write the answer into a JSON file 
# Returns a list of zeros and ones. 1 in first position means true prediction for top-1
def top_k_answers(solutions, actual_revs_indexes, test_num):

  top_13510 = []
  k_values = [1, 3, 5, 10]
  true_pred = []   # 1 if predicted truly and 0 otherwise

  for k in k_values:
    found_flag = False
    for solution in solutions:  # search in solution set
      if not found_flag:

        # total_top_k is a set of top k reviewers for all files in PR
        total_top_k = {r for f in range(P) for r in range(M) if 1 <= solution[f][r] <= k}  
        # print(total_top_k)

        top_ks = set(itertools.combinations(total_top_k, k))   # top_ks is exactly all possible k best reviewers as predicted answer for PR (all subsets with size k of total_top_k)
        # print(top_ks)

        for top_k in top_ks:   # top k predicted answers (each of them is the predicted answer)
          if actual_revs_indexes.issubset(set(top_k)) or set(top_k).issubset(actual_revs_indexes):   # this reviewer is in the correct solution of this test
            # answer found
            top_13510.append([all_reviewers[t] for t in set(top_k)])
            true_pred.append(1)
            print("top", k, "reviewers found correctly in index:", set(top_k))
            found_flag = True  # stop searching in other solutions
            break  # no need to search in other files

    # answer not found for this k
    if not found_flag:
      top_13510.append([all_reviewers[t] for t in set(top_k)])
      if actual_revs_indexes.issubset(set(top_k)): 
        true_pred.append(1)  # answer found
        print("top", k, "reviewers found correctly in index:", set(top_k))
      else:
        true_pred.append(0)

    print("up to k", k, top_13510)

  save_predictions(top_13510, test_num)   # write predicted answers into file
  return true_pred


In [ ]:
# Function to calculate Precision

In [ ]:
# Function to calculate Recall

In [ ]:
# # 20 and 600  -> 39 mins
# k = 1  # top-1
# for solution in solutions:  # search in solution set
#   top_k = set()
#   for f in range(0, P):     # solution for each file
#     for r in range(0, M):   # each rank for each file
#       if 1 <= solution[f][r] <= k:
#         top_k.add(r)
#     # print(top_k)
#     if len(top_k) != 0 and ((all_revs_indexes.issubset(top_k)) or top_k.issubset(all_revs_indexes)):   # this reviewer is in the correct solution of this test
#       print("found in:", solution)
#       break  # no need to search in other files

### main

In [ ]:
for test_num in range(0, 100):  # number of examination can be 0 to 644

  print("************ Test", test_num, "************")
  # creating test set
  test = total_test[test_num]   # one PR is test in each examination
  # creating train set
  if test_num == 0:  # must be 0 ********** change it for each run for now **********
    train = init_train
    # train.extend(total_test[0:test_num])  # must romved
  else:
    train.append(total_test[test_num-1])

  # test = total_test[1]
  # train = init_train[0:5901]

  # other calculations
  files = list(get_files(test))
  P = len(files)
  # print(files, "P is:", P)

  all_reviewers = list(get_all_reviewers(train))
  M = len(all_reviewers)
  # print(all_reviewers, "M is:", M)

  all_devs = list(get_all_devs(train))
  N = len(all_devs)
  # print(all_devs, "N is:", N)

  # Extract 3 metrices just one time
  FR = cal_expertise(files, all_reviewers, train)          # File-Reviewer matrix
  DR = cal_collaboration(all_devs, all_reviewers, train)   # Developer-Reviwer matrix
  A = cal_availability(all_reviewers, train, test['time']) # Availability vector
  FD = cal_fileDev(all_devs, files, train)                 # File-Developer matrix

  # Run Genetic on test PR
  # pop_size: test for 10, 20, 30, 40, 50
  # max_gen : stopping criterion = 100,000 fitness evaluations
  # cross_prob: crossover probability
  # mute_prob : mutation probabillity
  # gene_modif_prob: probability of gene modification
  if len(test['filePaths']) >= 6:
    solutions = genetic(P, M, N, files, all_reviewers, all_devs, FR, DR, A, FD, pop_size=10, max_gen=200, cross_prob=0.5, mute_prob=0.4, gene_modif_prob=0.2)
  else:
    solutions = genetic(P, M, N, files, all_reviewers, all_devs, FR, DR, A, FD, pop_size=30, max_gen=100, cross_prob=0.5, mute_prob=0.4, gene_modif_prob=0.2)

  # Run top_k_answers function for k=1, 3, 5, 10
  print("************ Results ************")
  actual_revs_indexes = {all_reviewers.index(r) if r in all_reviewers else -1 for r in test['reviewersId']}
  if -1 in actual_revs_indexes:
    actual_revs_indexes.remove(-1)
  if len(actual_revs_indexes) != 0:
    true_preds = top_k_answers(solutions, actual_revs_indexes, test_num)
    print("True predictions for test number", test_num, "are:", true_preds)
  else:   # reviewers of this test are all new people
    top_k_answers(solutions, actual_revs_indexes, test_num)
    true_preds = [0, 0, 0, 0]
  save_true_pred(true_preds)

  # Calculate precision and recall
  # TODO

************ Test 99 ************
Generation number: 0
Generation number: 10
Generation number: 20
Generation number: 30
Generation number: 40
Generation number: 50
Generation number: 60
Generation number: 70
Generation number: 80
Generation number: 90
Final set of solutions: [array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0

# Simple test

In [ ]:
test = {'changeId': '4',
        'filePaths': ['a.py', 'c.py'],
        'ownerId': '3',
        'reviewersId': ['0', '4'],
        'time': '2012-05-11 22:22:22'}

train = [{'changeId': '0',
        'filePaths': ['a.py', 'b.py'],
        'ownerId': '2',
        'reviewersId': ['0'],
        'time': '2012-05-9 22:22:22'},
        {'changeId': '1',
        'filePaths': ['c.py'],
        'ownerId': '3',
        'reviewersId': ['4'],
        'time': '2012-05-10 22:22:22'}]
        
for pr in train:
  pr['time'] = parser.parse(pr['time'])
test['time'] = parser.parse(test['time'])

# other calculations
files = ['c.py', 'a.py']
P = 2

all_reviewers = ['0', '4']
M = 2

all_devs = ['3', '2']
N = 2

# Extract 3 metrices just one time
FR = array([[0, 1],[1, 0]])       
DR = array([[0, 1],[2, 0]]) 
A = [2, 1]
FD = array([[1, 0],[0, 1]])
print("Shape of FR matrix is:", FR.shape)
print(FR)
print("Shape of DR matrix is:", DR.shape)
print(DR)
print("Size of A matrix is:", len(A))
print(A)
print("Shape of FD matrix is:", FD.shape)
print(FD)

Shape of FR matrix is: (2, 2)
[[0 1]
 [1 0]]
Shape of DR matrix is: (2, 2)
[[0 1]
 [2 0]]
Size of A matrix is: 2
[2, 1]
Shape of FD matrix is: (2, 2)
[[1 0]
 [0 1]]


In [ ]:
solutions = genetic(P, M, N, files, all_reviewers, all_devs, FR, DR, A, FD, pop_size=10, max_gen=500, cross_prob=0.5, mute_prob=0.4, gene_modif_prob=0.2)

Generation number: 0
Generation number: 1
Generation number: 2
Generation number: 3
Generation number: 4
Generation number: 5
Generation number: 6
Generation number: 7
Generation number: 8
Generation number: 9
Generation number: 10
Generation number: 11
Generation number: 12
Generation number: 13
Generation number: 14
Generation number: 15
Generation number: 16
Generation number: 17
Generation number: 18
Generation number: 19
Generation number: 20
Generation number: 21
Generation number: 22
Generation number: 23
Generation number: 24
Generation number: 25
Generation number: 26
Generation number: 27
Generation number: 28
Generation number: 29
Generation number: 30
Generation number: 31
Generation number: 32
Generation number: 33
Generation number: 34
Generation number: 35
Generation number: 36
Generation number: 37
Generation number: 38
Generation number: 39
Generation number: 40
Generation number: 41
Generation number: 42
Generation number: 43
Generation number: 44
Generation number: 4

In [ ]:
actual_revs_indexes = {all_reviewers.index(r) for r in test['reviewersId']}
print(actual_revs_indexes)
top_k_answers(solutions, actual_revs_indexes, 0)

{0, 1}
top 1 reviewers are in index: {1}
[['4']]
[['4'], ['4']]
[['4'], ['4'], ['4']]
[['4'], ['4'], ['4'], ['4']]


False

In [ ]:
R = [array([[1,0],[0,1]]),
     array([[2,1],[1,2]]),
     array([[1,2],[1,2]]),
     array([[2,1],[0,1]]),
     array([[1,2],[2,1]]),
     array([[1,2],[0,1]]),
     array([[0,1],[1,0]]),
     array([[1,0],[2,1]])]
R

[array([[1, 0],
        [0, 1]]), array([[2, 1],
        [1, 2]]), array([[1, 2],
        [1, 2]]), array([[2, 1],
        [0, 1]]), array([[1, 2],
        [2, 1]]), array([[1, 2],
        [0, 1]]), array([[0, 1],
        [1, 0]]), array([[1, 0],
        [2, 1]])]

In [ ]:
pop_size=4
total_pop=2*pop_size
max_gen=1
cross_prob=0.5
mute_prob=0.4
gene_modif_prob=0.2

In [ ]:
# Calculate fitness values for all genes in population
Availability_values = [fit_avail(A, R[i]) for i in range(0, total_pop)]
print("Avails:", Availability_values)

Avails: [0.3333333333333333, 0.1111111111111111, 0.125, 0.16666666666666666, 0.1111111111111111, 0.2, 0.3333333333333333, 0.14285714285714285]


In [ ]:
Expertise_values = [fit_expert(FR, R[i]) for i in range(0, total_pop)]
print("Expertise:", Expertise_values)

Expertise: [0.0, 2.0, 1.5, 1.0, 1.0, 0.5, 2.0, 0.5]


In [ ]:
Collaboration_values = [fit_collab(P, M, N, DR, FD, R[i]) for i in range(0, total_pop)]
print("Collaboration:", Collaboration_values)

Collaboration: [0, 3, 3, 1, 3, 1, 3, 2]


In [ ]:
# Fast non-dominated sort
F = fast_non_dominated_sort(Availability_values, Expertise_values, Collaboration_values)
print("Fronts:", F)

Fronts: [[0, 3, 5, 6], [7, 1, 2], [4]]


In [ ]:
P_new = []    # new parent population
i = 0
while len(P_new) + len(F[i]) <= pop_size:
  F[i] = sort_front_by_dist(Availability_values, Expertise_values, Collaboration_values, F[i])
  P_new.extend(F[i])   # union of new parents and current front
  i += 1
  print("new parents:", P_new)

new parents: [0, 3, 6, 5]


In [ ]:
# Apply crowding distance on F[i] only. because this front should be splitted
F[i] = sort_front_by_dist(Availability_values, Expertise_values, Collaboration_values, F[i])
F[i]

[7, 1, 2]

In [ ]:
# The first pop_size-|P_new| elements of F_i are chosen
remain = pop_size - len(P_new)
parent_indexes = P_new + F[i][0:remain]
print("Selected parent numbers for next generation:", parent_indexes)

# corresponding genes of gene numbers kept as parents for next generation
P0 = [R[idx] for idx in parent_indexes]
print("Next solutions:", P0)

Selected parent numbers for next generation: [0, 3, 6, 5]
Next solutions: [array([[1, 0],
       [0, 1]]), array([[2, 1],
       [0, 1]]), array([[0, 1],
       [1, 0]]), array([[1, 2],
       [0, 1]])]
